In [11]:
import pandas as pd
from datetime import datetime

def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

df = pd.read_csv("Tokopedia_AppStore 13127 AppStore Indonesian.csv", header=0, parse_dates=[0], index_col=0, date_parser=parser)

In [2]:
import torch
from transformers import BertTokenizer, BertModel
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Loading the pre-trained BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
summaries = {}
originals = []
summary = []

In [12]:
df.head()

review
date                                                  
2023-05-31 23:58:56                             Mantap
2023-05-31 23:58:22  Reliable apps for online shopping
2023-05-31 23:58:21                               Nice
2023-05-31 23:50:09                               good
2023-05-31 23:39:48                              Cepet

In [13]:
# Convert the index to a monthly period
df.index = df.index.to_period('M')

# Group the dataframe by month
grouped_df = df.groupby(df.index)

# Iterate over the groups and access the data for each month
for month, data in grouped_df:
    print(f"Month: {month}")


Month: 2023-02
Month: 2023-03
Month: 2023-04
Month: 2023-05


In [ ]:
count = 1
for text in df['review']:
    # Preprocessing the text
    if isinstance(text, str):  # Check if text is a string
        sentences = sent_tokenize(text)
        if len(sentences) < 2:
            summary.append(text)
        else:
            tokenized_sentences = [tokenizer.encode(sent, add_special_tokens=True) for sent in sentences]

            # Encoding the input
            max_len = 0
            for i in tokenized_sentences:
                if len(i) > max_len:
                    max_len = len(i)

            padded_sentences = []
            for i in tokenized_sentences:
                while len(i) < max_len:
                    i.append(0)
                padded_sentences.append(i)

            input_ids = torch.tensor(padded_sentences)

            # Generating the sentence embeddings
            with torch.no_grad():
                last_hidden_states = model(input_ids)[0]

            sentence_embeddings = []
            for i in range(len(sentences)):
                sentence_embeddings.append(torch.mean(last_hidden_states[i], dim=0).numpy())

            # Summarizing the text
            # Compute the similarity matrix
            similarity_matrix = cosine_similarity(sentence_embeddings)

            # Generate the summary
            num_sentences = 1
            summary_sentences = []
            for i in range(num_sentences):
                sentence_scores = list(enumerate(similarity_matrix[i]))

            sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)
            summary_sentences.append(sentences[sentence_scores[1][0]])

            summarization = ' '.join(summary_sentences)
            
            summary.append(summarization)
            
    else:
        summary.append(text)

    originals.append(text)

    if count % 1000 == 0:
        print(f"{count} DONE")
    count += 1

1000 DONE
2000 DONE
3000 DONE
4000 DONE
5000 DONE
6000 DONE
7000 DONE
8000 DONE
9000 DONE
10000 DONE
11000 DONE
12000 DONE


In [19]:
for month, data in grouped_df:
    text = ' '.join(data['review'].astype(str).tolist())

    # Preprocessing the text

    sentences = sent_tokenize(text)
    tokenized_sentences = [tokenizer.encode(sent, add_special_tokens=True) for sent in sentences]

    # Encoding the input
    max_len = 0
    for i in tokenized_sentences:
        if len(i) > max_len:
            max_len = len(i)

    padded_sentences = []
    for i in tokenized_sentences:
        while len(i) < max_len:
            i.append(0)
        padded_sentences.append(i)

    input_ids = torch.tensor(padded_sentences)

    # Generating the sentence embeddings
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]

    sentence_embeddings = []
    for i in range(len(sentences)):
        sentence_embeddings.append(torch.mean(last_hidden_states[i], dim=0).numpy())

    # Summarizing the text
    # Compute the similarity matrix
    similarity_matrix = cosine_similarity(sentence_embeddings)

    # Generate the summary
    num_sentences = 1
    summary_sentences = []
    for i in range(num_sentences):
        sentence_scores = list(enumerate(similarity_matrix[i]))

    sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)
    summary_sentences.append(sentences[sentence_scores[1][0]])

    summarization = ' '.join(summary_sentences)

    summary.append(summarization)

    originals.append(text)
    print(f"{month} DONE")

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 9177976752 bytes.

In [6]:
text = ' '.join(df['review'].astype(str).tolist())

# Preprocessing the text

sentences = sent_tokenize(text)
tokenized_sentences = [tokenizer.encode(sent, add_special_tokens=True) for sent in sentences]

# Encoding the input
max_len = 0
for i in tokenized_sentences:
    if len(i) > max_len:
        max_len = len(i)

padded_sentences = []
for i in tokenized_sentences:
    while len(i) < max_len:
        i.append(0)
    padded_sentences.append(i)

input_ids = torch.tensor(padded_sentences)

# Generating the sentence embeddings
with torch.no_grad():
    last_hidden_states = model(input_ids)[0]

sentence_embeddings = []
for i in range(len(sentences)):
    sentence_embeddings.append(torch.mean(last_hidden_states[i], dim=0).numpy())

# Summarizing the text
# Compute the similarity matrix
similarity_matrix = cosine_similarity(sentence_embeddings)

# Generate the summary
num_sentences = 1
summary_sentences = []
for i in range(num_sentences):
    sentence_scores = list(enumerate(similarity_matrix[i]))

sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)
summary_sentences.append(sentences[sentence_scores[1][0]])

summarization = ' '.join(summary_sentences)

summary.append(summarization)

originals.append(text)

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 104643376128 bytes.

In [ ]:
summaries['Original'] = originals
summaries['Summary'] = summary

df_summaries = pd.DataFrame(summaries)

df_summaries.head()

In [ ]:
len(df_summaries)

In [20]:
df_summaries.to_csv("1 facebook appstore summarization.csv")